In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
data_iris = load_iris()                         # アヤメのデータセットの読み込み

# print(data_iris.feature_names)                # 説明変数の表示
# print(data_iris.target_names)                 # 目的変数の表示
# print(data_iris['DESCR'])                     # 変数の説明の表示

scaler = StandardScaler()

x_train, x_test, y_train, y_test = train_test_split(data_iris.data, data_iris.target, test_size=0.2, random_state=0)

x_train_reshaped = x_train.reshape(-1, 4)           # 本来は必要なし。元々 1 次元ベクトル
x_train_normalized = scaler.fit_transform(x_train_reshaped)
y_train_categorized = np.eye(3)[y_train]

x_test_reshaped = x_test.reshape(-1, 4)             # 本来は必要なし。元々 1 次元ベクトル
x_test_normalized = scaler.fit_transform(x_test_reshaped)
y_test_categorized = np.eye(3)[y_test]

print(x_train_reshaped.shape)
print(y_train_categorized.shape)

(120, 4)
(120, 3)


In [ ]:
#------------------------------------------------------------------------------
# Neural Network
#------------------------------------------------------------------------------
class NeuralNetwork:

  def __init__(self, input_size, hidden_size, output_size):

    self.hidden_weight = np.random.randn(input_size, hidden_size) * np.sqrt(2.0 / input_size)          # He 初期化
    self.hidden_bias = np.zeros((1, hidden_size))

    self.output_weight = np.random.randn(hidden_size, output_size) * np.sqrt(2.0 / hidden_size)        # He 初期化
    self.output_bias = np.zeros((1, output_size))

  def func_relu(self, x):
    y = np.maximum(0, x)
    return y

  def func_relu_deriv(self, x):
    y = np.where(x > 0, 1, 0)
    return y

  def func_softmax(self, x):
    exp_x = np.exp(x - np.max(x))
    y = exp_x / np.sum(exp_x, axis=1, keepdims=True)
    return y

  def func_forward(self, x_data):

    ans = np.dot(x_data, self.hidden_weight) + self.hidden_bias
    hidden_data = self.func_relu(ans)

    ans = np.dot(hidden_data, self.output_weight) + self.output_bias
    output_data = self.func_softmax(ans)

    return hidden_data, output_data

  def func_backward(self, x_data, y_data, hidden_data, output_data):

    learning_rate = 0.1

    output_loss = output_data - y_data
    output_weight = hidden_data.T.dot(output_loss)
    output_bias = np.sum(output_loss, axis=0)

    hidden_loss = output_loss.dot(self.output_weight.T) * self.func_relu_deriv(hidden_data)
    hidden_weight = x_data.T.dot(hidden_loss)
    hidden_bias = np.sum(hidden_loss, axis=0)

    self.output_weight -= learning_rate * output_weight / len(x_data)
    self.output_bias -= learning_rate * output_bias / len(x_data)

    self.hidden_weight -= learning_rate * hidden_weight / len(x_data)
    self.hidden_bias -= learning_rate * hidden_bias / len(x_data)

  def func_train(self, x_data, y_data):

    batch_size = 10
    epochs = 100

    for e in range(epochs):
      for i in range(0, len(x_data), batch_size):
        x_batch = x_data[i: i + batch_size]
        y_batch = y_data[i: i + batch_size]
        hidden_data, output_data = self.func_forward(x_batch)
        self.func_backward(x_batch, y_batch, hidden_data, output_data)

      _, output_data = self.func_forward(x_data)
      acc_score = np.mean(np.argmax(output_data, axis=1) == np.argmax(y_data, axis=1))
      loss_value = np.mean(-y_data * np.log(output_data))
      print(f'Epoch {e + 1}, Loss: {loss_value:.4f}, Accuracy: {acc_score:.4f}')

  def func_predict(self, x):
    _, output_data = self.func_forward(x)
    return np.argmax(output_data, axis=1)

#------------------------------------------------------------------------------
# Main
#------------------------------------------------------------------------------
neural_network = NeuralNetwork(4, 10, 3)
neural_network.func_train(x_train_normalized, y_train_categorized)

y_pred = neural_network.func_predict(x_test_normalized)
acc_score = np.mean(y_pred == y_test)

print(f'Accuracy Score: {acc_score:.4f}')

print(y_test[:10])
print(y_pred[:10])

Epoch 1, Loss: 0.1362, Accuracy: 0.8500
Epoch 2, Loss: 0.1162, Accuracy: 0.8667
Epoch 3, Loss: 0.1036, Accuracy: 0.8750
Epoch 4, Loss: 0.0940, Accuracy: 0.8750
Epoch 5, Loss: 0.0864, Accuracy: 0.8917
Epoch 6, Loss: 0.0801, Accuracy: 0.9167
Epoch 7, Loss: 0.0747, Accuracy: 0.9333
Epoch 8, Loss: 0.0701, Accuracy: 0.9417
Epoch 9, Loss: 0.0659, Accuracy: 0.9417
Epoch 10, Loss: 0.0622, Accuracy: 0.9417
Epoch 11, Loss: 0.0586, Accuracy: 0.9583
Epoch 12, Loss: 0.0555, Accuracy: 0.9583
Epoch 13, Loss: 0.0526, Accuracy: 0.9583
Epoch 14, Loss: 0.0501, Accuracy: 0.9583
Epoch 15, Loss: 0.0478, Accuracy: 0.9583
Epoch 16, Loss: 0.0457, Accuracy: 0.9583
Epoch 17, Loss: 0.0439, Accuracy: 0.9583
Epoch 18, Loss: 0.0423, Accuracy: 0.9583
Epoch 19, Loss: 0.0408, Accuracy: 0.9583
Epoch 20, Loss: 0.0394, Accuracy: 0.9583
Epoch 21, Loss: 0.0381, Accuracy: 0.9583
Epoch 22, Loss: 0.0370, Accuracy: 0.9583
Epoch 23, Loss: 0.0359, Accuracy: 0.9583
Epoch 24, Loss: 0.0349, Accuracy: 0.9583
Epoch 25, Loss: 0.0339, A